# Lab Sheet 3a: More Word Frequency Vectors with Spark

These tasks are for working in the lab session and during the week.

We will use the **same data as in week 2** (18 files in "/content/drive/My Drive/Big_Data/data/library") and use **some more RDD functions**. We will apply **two different approaches** to create and use **fixed size vectors**.



First, run the usual preparations.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# install spark
%cd
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # installing java
!tar -xzf "/content/drive/My Drive/Big_Data/data/spark/spark-3.5.0-bin-hadoop3.tgz" # unpacking
import os # Python package for interaction with the operating system
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" # tell the system where Java lives
os.environ["SPARK_HOME"] = "/root/spark-3.5.0-bin-hadoop3" # and where spark lives
!pip install -q findspark # install helper package
import findspark # use the helper package
findspark.init() # to set up spark
%cd "/content/drive/My Drive/Big_Data"

import pyspark
# get a spark context
sc = pyspark.SparkContext.getOrCreate()
print(sc)
# and a spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/root
/content/drive/MyDrive/Big_Data_Material/Big_Data
<SparkContext master=local[*] appName=pyspark-shell>


In [ ]:
!ls /content/drive/My\ Drive/Big_Data/data/spark

 data					  'Lab 2 - Extracting Word Freq Vectors.ipynb'
'Lab 1 - Word Counting with Spark.ipynb'   solutions


Here is **code from week 2** that we **run first**, and then extend.

In [ ]:
import re
import os.path

def stripFinalS( word ):
    word = word.lower() # lower case
    word = word.rstrip('s')
    return word

def splitFileWords(filenameContent): # your splitting function
    f, c = filenameContent # unpack the input tuple
    fwLst = [] # the new list for (filename,word) tuples
    wLst = re.split('\W+',c) # <<< now create a word list wLst by splitting c (the content)
    for w in wLst : # iterate through the list
        fwLst.append((f, stripFinalS(w))) # and append (f,w) to the fwLst
    return fwLst # return a list of (f,w) tuples

from pyspark import SparkContext

sc = SparkContext.getOrCreate()

dirPath = "/content/drive/My Drive/Big_Data/data/library" #  path
ft_RDD = sc.wholeTextFiles(dirPath) # create an RDD with wholeTextFiles
# just take filenames, drop path and extension for readability
fnt_RDD = ft_RDD.map(lambda ft: (os.path.splitext(os.path.basename(ft[0]))[0], ft[1]))
fw_RDD1 = fnt_RDD.flatMap(splitFileWords) # split words per file, strip final 's'
fw_RDD = fw_RDD1.filter(lambda fw: fw[1] not in ['','project','gutenberg', 'ebook']) # get rid of some unwanted words
fw_RDD.take(3)
# output should look like this: [('emma', 'the'), ('emma', 'of'), ('emma', 'emma')]

## 1) Warm-up
Let's start with a **few small tasks**, to become more fluent with **RDDs and lambda expressions**.

1. Count the **number of documents**.
2. Determine the number **distinct words** in total (the vocabulary size) using `RDD.distinct()`. This involves **removing the filenames** from the `(f, w)` pairs and getting the RDD size (with `RDD.count()`).
3. Get the number of **words** (including repeated ones) per book.
4. Determine the number of distinct words per book. This involves determining the distinct `(f,w)` pairs, getting a list of words per file, and getting the list size.
5. Count the average number of occurrences per word per file (words/vocabulary). Use `RDD.join()` to get both numbers into one RDD.

Remember that `>>>` indicates a line where you should do something - you need to remove it for any code to work.
Typically, you'll find a `...` placeholder in that line at the place where you should add the code.  

### a) Library size

In [ ]:
>>>print("Number of documents: ", ft_RDD ...) # count the number of docs

### b) Vocabulary size

In [ ]:
>>>w_RDD = fw_RDD.map(...) # remove the file names, keep just the words
>>>w_RDDu = w_RDD ... # keep only one unique instance of every word
print('Total vocabulary size: ', w_RDDu.count())

### c) Words per book

In [ ]:
from operator import add
>>>f1_RDD = fw_RDD.map(lambda fw: ...) # wrap (f, w) to (f, 1)
>>>fc_RDD = f1_RDD.reduceByKey(...) # add up the 1s
print('Words per book: ', fc_RDD.take(3))
# extra task: try also to express this with just one function that appeared in the lecture last week
>>> print('Words per book (v2): ', fw_RDD ...)

### d) Vocabulary per book

In [ ]:
# we can reuse the same solution as above, if we do one thing before ...
>>> fw_RDDu = ...
f1_RDDu = fw_RDDu.map(lambda fw: (fw[0], 1)) # wrap (f, w) to (f, 1)
fcu_RDD = f1_RDDu.reduceByKey(add) # add up the 1s
print('Vocabulary per book: ', fcu_RDD.take(3))
# extra task: replacing the map and reduce by one function as in c)
>>> print('Vocabulary per book (v2): ', fw_RDDu ... )

### e) Average occurrences of words per book (i.e. words/vocab per book)

In [ ]:
>>>f_wv_RDD = fc_RDD ... fcu_RDD # join the two RDDs to get (f, (w, v)) tuples
print(f_wv_RDD.take(3))
>>>f_awo_RDD = f_wv_RDD.map(lambda f_wv: (f_wv[0], ... )) # this is the tricky part.
            # Resolve nested tuples in the lambda to get (filename, words / vocab) tuples
print('Average word occurrences: ',f_awo_RDD.take(3))
# should look like this [('henry_V', 6.212341574901309), ('macbeth', 5.699808271706382), ('lady_susan', 8.430416532127866)]


## 2) Fixed vectors: Reduced vocabulary approach

The first task in this lab is to use a **reduced vocabulary** - **only stopwords** from a list This is a common approach in **stylometry**.
It also make sure that we have a **fixed size vector**, which is needed for most machine learning task.

A problem is that some **stopwords** might **not appear in some documents**. We will deal with that by creating an RDD with `((f, w), 0)` tuples that we then merge with the `((f, w), count)` RDD.

Start by running the code above, then you can add 1s and use `reduceByKey(add)` like last week to get the counts of the words per filename.

Then, please make sure that all stopwords are present by creating a new RDD that contains the keys of the fw_RDD, i.e. the filenames, using the `keys()` method of the RDD. Then you can use `flatMap` to create a `[((filename, stopword), 0), ...]` list, using a list comprehension. The 0s should not be 1s, as we don't want add to add extra counts.
The RDD with `((filename, stopword), 0)` tuples can then be merged with `fw_RDD2` using `union()`. Then you can count as normal.

In [ ]:
from operator import add

stopwlst = ['the','a','in','of','on','at','for','by','i','you','me'] # stopword list
>>>fw_RDD2 = fw_RDD.filter(lambda fw: ... ) # filter, keeping only stopwords

# create a list of stopwords with count 0 to avoid missing elements in our vectors
fw_0_RDD = fw_RDD.keys().flatMap(lambda f: [((f, sw), 0) for sw in stopwlst])
print(fw_0_RDD.take(3))
# output should look like this:
#[(('emma', 'the'), 0), (('emma', 'a'), 0), (('emma', 'in'), 0)]

>>>fw_1_RDD = fw_RDD2.map(lambda fw: ...)  # <<< change (f,w) to ((f,w),1)
print(fw_1_RDD.take(3))
# output should look like this:
#[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]

>>>fw_10_RDD = fw_1_RDD ... fw_0_RDD # <<< create the union on the two RDDs
print(fw_10_RDD.take(3))
# output should look like this:
#[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]

fw_c_RDD = fw_10_RDD.reduceByKey(add) # count the words
print(fw_c_RDD.take(3))
# output should look like this:
#[(('emma', 'the'), 5380), (('emma', 'by'), 591), (('emma', 'you'), 2068)]

## 3) Creating sorted lists

As a next step, map the `((filename, word), count)` to `( filename, [(word, count)])` using the function `reGrpLst` to regroup and create a list.

Then sort the `[(word, count), ...]` lists in the values (i.e. 2nd part of the tuple) with the the words as keys. Have a [look at the Python docs](https://docs.python.org/3.6/library/functions.html?#sorted) for how to do this. Hint: use a lambda that extracts the words as the key, e.g. `sorted(f_wcL[1], key = lambda wc: ... )`.   

In [ ]:
def reGrpLst(fw_c): # we get a nested tuple
    >>>     # split the outer tuple
    >>>     # split the inner tuple
    return (f,[(w,c)]) # return (f,[(w,c)]) structure. Can be used verbatim, if your variable names match.


>>>f_wcL_RDD = fw_c_RDD.map(...) # apply reGrpLst
f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) # create [(w,c), ... ,(w,c)] lists per file
>>>f_wcLsort_RDD = f_wcL2_RDD.map(lambda f_wcL: (f_wcL[0], sorted(...))) #<<< sort the word count lists by word
print(f_wcLsort_RDD.take(3))
# output:
# [('macbeth', [('a', 395), ('at', 64), ('by', 74), ('for', 142), ('i', 581), ('in', 227), ('me', 119), ('of', 427), ...
>>>f_wVec_RDD = f_wcLsort_RDD.map(lambda f_wc: (f_wc[0], [c for ...])) # remove the words from the wc pairs
f_wVec_RDD.take(3)
# output:
# [('macbeth', [395, 64, 74, 142, 581, 227, 119, 427, 76, 765, 272]),
#  ('lady_susan', [611, 161, 152, 262, 1106, 402, 200, 787, 140, 784, 353]),
#  ('merchant_of_venice', [646, 75, 131, 254, 976, 319, 260, 535, 81, 938, 507])]

## 4) Clustering

Now we have feature vectors of fixed size and fixed sequential order, we can use KMeans as provided by Spark.

The files in our library are by two authors. After clustering, check if the clusters reflect authorship:

WILLIAM SHAKESPEARE:
merchant_of_venice,
richard_III,
midsummer,
tempest,
romeoandjuliet,
othello,
henry_V,
macbeth,
king_lear,
julius_cesar,
hamlet

JANE AUSTEN:
mansfield_park,
emma,
northanger_abbey,
lady_susan,
persuasion,
prideandprejudice,
senseandsensibility

In [ ]:
from math import sqrt
from pyspark.mllib.clustering import KMeans

>>>wVec_RDD = f_wVec_RDD.map(lambda f_wcl: ...) # strip the filenames, keep only the vectors

# Build the model (cluster the data)
clusterModel = KMeans.train(wVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the filenames to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fc_RDD.collect():
    print(s)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusterModel.centers[clusterModel.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = wVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))
# now check if the clusters match the authors
# output:
# ('macbeth', 0)
# ('lady_susan', 0)
# ('merchant_of_venice', 0)
# ('othello', 0)
# ('persuasion', 1)
# ('emma', 1)

## 5) Alternative approach: feature hashing

Instead of the previous approach, we now use feature hashing, as done last week.

In [ ]:
def hashing_vectorizer(word_count_list, N):
    v = [0] * N  # create fixed size vector of 0s
    for word_count in word_count_list:
>>>        ... # unpack tuple
        h = hash(word) # get hash value
        v[h % N] = v[h % N] + count # add count
    return v # return hashed word vector

from operator import add

N = 10

# we use fw_RDD from the beginning with all the words, not just stopwords
>>>fw_1_RDD = fw_RDD.map(lambda fw: ...)  # <<< change (f,w) to ((f,w),1)
fw_c_RDD = fw_1_RDD.reduceByKey(add) # as above
f_wcL_RDD = fw_c_RDD.map(reGrpLst) # as above
f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) # create [(w,c), ... ,(w,c)] lists per file
>>>f_hwVec_RDD = f_wcL2_RDD.map(lambda f_wcL: (f_wcL[0], ...)) # apply the hashing_vectorizer to the word-count list
print(f_hwVec_RDD.take(3))
# output:
# [('henry_V', [3176, 3044, 2429, 2813, 2947, 3257, 2144, 4706, 1823, 3561]), ('macbeth', [1962, 2023, 1875, 2145, 2011, 2238, 1610, 3166, 1343, 2437]), ('lady_susan', [3491, 2624, 1871, 2726, 2847, 2896, 1967, 3847, 1179, 2661])]

In [ ]:
from math import sqrt
from pyspark.mllib.clustering import KMeans

hwVec_RDD = f_hwVec_RDD.map(lambda f_wcl: f_wcl[1]) # strip the filenames

# Build the model (cluster the data)
clusterModel = KMeans.train(hwVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fhc_RDD = f_hwVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fhc_RDD.collect():
    print(s)

# resusing 'error' function from above
WSSSE = hwVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within-Set Sum of Squared Error = " + str(WSSSE))

## 6) Compensating document length: normalised vectors

**'Lady Susan'** ends up reliably in the **wrong cluster**. A possible explanation could be that it is **shorter** than the other Austen works. Try **normalising** the word counts, i.e. by dividing by their sum. That takes away the effect of length. Is there and effect on the clustering?
    
You can use a list comprehension for the normalisation.

In [ ]:
>>>nwVec_RDD = wVec_RDD.map(lambda v: ...) # provide a list comprehension that
                            # normalises the values by dividing by the sum over the list
print("Normalised vectors: ", nwVec_RDD.take(3))
# output:
# Normalised vectors:  [[0.12563613231552162, 0.020356234096692113, 0.023536895674300253, 0.045165394402035625, 0.18479643765903309, ...

# Build the model (cluster the data)
clusterModel = KMeans.train(nwVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fnc_RDD = f_wVec_RDD.map(lambda fv: (fv[0], clusterModel.predict(fv[1])))
for s in fnc_RDD.collect():
    print(s)
# output
# ('macbeth', 0)
# ('lady_susan', 0)
# ('merchant_of_venice', 0)
# ..

**Comment:** Unfortunately, there is no positive effect on the classification of lady_susan. We'll need to use another method, e.g. supervised learning, to determine authorship.

## 7) Building an index

Starting from the fw_RDD we now start **building the index** and calculating the **IDF values**. Since we have the TF values already, we only need to keep the unique filenames per word using [RDD.distinct()](https://spark.apache.org/docs/2.4.4/api/python/pyspark.html#pyspark.RDD.distinct).  
Then we create a list of filenames. The length of the list is the **document frequency DF** per word.
From the DF value we can calculate the **IDF** value as **`log(num_documents/DF)`**.

In [ ]:
from math import log

num_documents = ft_RDD.count()

fwu_RDD = fw_RDD.distinct() # get unique file/word pairs
>>>wfl_RDD = fwu_RDD.map(lambda fw: (fw[1], ...)) # create (w, [f]) tuples
wfL_RDD = wfl_RDD.reduceByKey(add) # concatenate the lists with 'add'
print(wfL_RDD.take(3))
# output:
# [('of', ['henry_V', 'macbeth', 'lady_susan', 'midsummer', 'merchant_of_venice', 'king_lear', 'northanger_abbey', 'othello', ...

>>>wdf_RDD = wfL_RDD.map(lambda wfl: (wfl[0], ...)) # get the DF replacing the file list with its length
print("DF: ",wdf_RDD.take(3))
# output:
# DF:  [('of', 18), ('shakespeare', 15), ('henry', 9)]

>>>widf_RDD = wdf_RDD.map(lambda wdf: (wdf[0], ...)) # get the IDF by replacing DF with log(num_documents/DF)
print("IDF: ",widf_RDD.take(3))
# output:
# IDF:  [('of', 0.0), ('shakespeare', 0.1823215567939546), ('henry', 0.6931471805599453)]

We will work more with IDF values next week.